In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark=SparkSession.builder.master("local[1]").appName("ireland_hostipality_details.com").getOrCreate()

24/10/09 13:52:20 WARN Utils: Your hostname, AI-CJB-LAP-459 resolves to a loopback address: 127.0.1.1; using 192.168.1.164 instead (on interface wlp0s20f3)
24/10/09 13:52:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/09 13:52:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
hotel_details=spark.read.parquet("/home/ai/Documents/my files/assignment_2/Pyspark_Assignment_2/raw_data_in/hotelsourcesdetails.parquet")
hotel_rates=spark.read.parquet("/home/ai/Documents/my files/assignment_2/Pyspark_Assignment_2/raw_data_in/hospitality_raw_rates_20240922_123949.parquet")

In [4]:
# Alias the DataFrames
hd = hotel_details.alias("hd")
hr = hotel_rates.alias("hr")

In [5]:
hd=hd.withColumnRenamed("hotelid", "hd_hotelid")
hd=hd.withColumnRenamed("websitecode","hd_websitecode")

In [6]:
# Filter and merge data
irl_hd = hd.filter(hd.countrycode == "IRL")
# irl_hd = hd.withColumnRenamed("hotelid", "hd_hotelid")
irl_hd = irl_hd.withColumnRenamed("id", "hd_id")#column renamed to avoid the ambiguity
# irl_hd = hd.withColumnRenamed("hotelid", "hd_hotelid")
ireland_df = irl_hd.join(hr, irl_hd["hd_id"] == hr["id"], "inner")

In [7]:
ireland_df.show(5)

24/10/09 13:52:23 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+---------+----------+--------------+--------------------+--------------------+--------+--------+-----------+----+-----------+-----------+------+---------------+------+----------+------------+-----------+-------------------+---+------+--------------------+----------+-------+--------+--------------------+--------------------+--------------------+--------------------+---------+-------+------+----------+----------+--------+---------+----------+---------+--------------------+---------+---+--------------------+-------------+--------------------+--------------+-------+-----+-------+--------+-------+
|hd_id|hotelcode|hd_hotelid|hd_websitecode|           hotelname|             address|    city|   state|countrycode| zip|        lat|        lng|status|   propertytype|rating|larbarterm|reviewscount|websitecode|        dtcollected|los|guests|            roomtype|onsiterate|netrate|currency|     roomdescription|            rateplan|           sourceurl|       roomamenities|occupancy|ispromo|c

In [8]:
# Print dataset shape and summary statistics
print("Shape of the dataset:", ireland_df.count())
ireland_df.summary().show(5)
# ireland_df.select(hd.id,).describe().show()

Shape of the dataset: 31165


+-------+------------------+------------------+--------------------+------------------+--------------------+------------+--------+--------------------+-----------+-----------------+------------------+------------------+------------------+------------+------------------+----------+------------------+-----------+-----+------+--------------------+------------------+------------------+--------+---------------+--------------------+--------------------+--------------------+------------------+-------+------+----------+----------+------------------+---------+------------------+-------------------+------------------+------------------+-----+--------------------+-------------+-------------+--------------+-------+------------------+------------------+------------------+-------+
|summary|             hd_id|         hotelcode|          hd_hotelid|    hd_websitecode|           hotelname|     address|    city|               state|countrycode|              zip|               lat|               lng|   